- look into architechture modification and add numerical values as concatenation

In [ ]:
%pip install sentence-transformers
%pip install umap-learn

In [ ]:
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
import umap
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import losses

In [ ]:
print(torch.cuda.device_count())

In [ ]:
df_data = pd.read_csv("drive/MyDrive/DATA_transactions/2024_05_20_sample_pairs.csv", index_col = None)
print(df_data.shape)

In [ ]:
df_validate = df_data[0:100]
df_test = df_data[100:200]
df_train = df_data[200:]

In [ ]:
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader

In [ ]:

word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
df_data['label'] = df_data['label'].astype('float32')
df_data['true_label'] = df_data['true_label'].astype('float32')
train_examples = [InputExample(texts = [df_data.loc[i,'sentence'], df_data.loc[i,'merchant_name_combined']], label=df_data.loc[i,'true_label']  ) for i in range(df_data.shape[0])]

In [ ]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size = 4)

In [ ]:
train_loss = losses.CosineSimilarityLoss(model)

## Tune the model

In [ ]:
model.fit(train_objectives = [(train_dataloader, train_loss)], epochs =5 , warmup_steps = 100)

## Evaluation

In [ ]:
validate_data = df_data.sample(30, random_state = 100)
trx_descriptions = validate_data['sentence'].tolist()
merchants = validate_data['merchant_name_combined'].tolist()
trx_embeddings = model.encode(trx_descriptions)
merchants_embeddings = model.encode(merchants)

In [ ]:
print(trx_embeddings.shape)
print(merchants_embeddings.shape)

In [ ]:
## look into numba
cosine_similarity(trx_embeddings, merchants_embeddings).shape
validate_data['cos_similarity']=np.diag(cosine_similarity(trx_embeddings, merchants_embeddings))

In [ ]:
validate_data[['label', 'true_label','cos_similarity']]

In [ ]:
# reducer = umap.UMAP(n_epochs=400,  n_neighbors=150, min_dist=0.1)
# reducer.fit(merchants_embeddings
# )
# embedding_2d = reducer.transform(merchants_embeddings)
# fig = px.scatter(embedding_2d, x=0, y=1,opacity=0.05, height=500, hover_name=names)
# fig.show()